In [1]:
import pandas as pd
data = pd.read_csv('./TUANDROMD.csv')

### EDA

Датасет представляет собой данные о приложениях (вирусных и безопасных), собранные университетом Тезпура (Индия).<br>
https://doi.org/10.1109/CICT51604.2020.9312053<br>

Фичи - бинарные флаги, заправшивало ли приложение разрешения и вызывало ли определенные методы системы.<br>
Таргет - класс (0/1), является ли приложение вирусным (1) или нет (0).<br>

Таким образом, задача - бинарная классификация, метрика - f1-мера.

Рассмотрим первые 10 строк:

In [2]:
data.head(10)

,ACCESS_ALL_DOWNLOADS,ACCESS_CACHE_FILESYSTEM,ACCESS_CHECKIN_PROPERTIES,ACCESS_COARSE_LOCATION,ACCESS_COARSE_UPDATES,ACCESS_FINE_LOCATION,ACCESS_LOCATION_EXTRA_COMMANDS,ACCESS_MOCK_LOCATION,ACCESS_MTK_MMHW,ACCESS_NETWORK_STATE,...,Landroid/telephony/TelephonyManager;->getLine1Number,Landroid/telephony/TelephonyManager;->getNetworkOperator,Landroid/telephony/TelephonyManager;->getNetworkOperatorName,Landroid/telephony/TelephonyManager;->getNetworkCountryIso,Landroid/telephony/TelephonyManager;->getSimOperator,Landroid/telephony/TelephonyManager;->getSimOperatorName,Landroid/telephony/TelephonyManager;->getSimCountryIso,Landroid/telephony/TelephonyManager;->getSimSerialNumber,Lorg/apache/http/impl/client/DefaultHttpClient;->execute,Label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


Посмотрим, сбалансированы ли классы.

In [3]:
data['Label'].value_counts()

Label
1.0    3565
0.0     899
Name: count, dtype: int64

Классы несбалансированы, что лишний раз подтверждает, что стоит использовать f1-меру, а не более простые метрики, например, accuracy.<br>
Посмотрим, есть ли пропуски в данных.

In [4]:
nans = {i:data[i].isna().sum() for i in data.columns}
sorted_nans = {k: v for k, v in sorted(nans.items(), key=lambda item: item[1])}
sorted_nans

{'ACCESS_ALL_DOWNLOADS': 1,
 'ACCESS_CACHE_FILESYSTEM': 1,
 'ACCESS_CHECKIN_PROPERTIES': 1,
 'ACCESS_COARSE_LOCATION': 1,
 'ACCESS_COARSE_UPDATES': 1,
 'ACCESS_FINE_LOCATION': 1,
 'ACCESS_LOCATION_EXTRA_COMMANDS': 1,
 'ACCESS_MOCK_LOCATION': 1,
 'ACCESS_MTK_MMHW': 1,
 'ACCESS_NETWORK_STATE': 1,
 'ACCESS_PROVIDER': 1,
 'ACCESS_SERVICE': 1,
 'ACCESS_SHARED_DATA': 1,
 'ACCESS_SUPERUSER': 1,
 'ACCESS_SURFACE_FLINGER': 1,
 'ACCESS_WIFI_STATE': 1,
 'activityCalled': 1,
 'ACTIVITY_RECOGNITION': 1,
 'ACCOUNT_MANAGER': 1,
 'ADD_VOICEMAIL': 1,
 'ANT': 1,
 'ANT_ADMIN': 1,
 'AUTHENTICATE_ACCOUNTS': 1,
 'AUTORUN_MANAGER_LICENSE_MANAGER': 1,
 'AUTORUN_MANAGER_LICENSE_SERVICE(.autorun)': 1,
 'BATTERY_STATS': 1,
 'BILLING': 1,
 'BIND_ACCESSIBILITY_SERVICE': 1,
 'BIND_APPWIDGET': 1,
 'BIND_CARRIER_MESSAGING_SERVICE': 1,
 'BIND_DEVICE_ADMIN': 1,
 'BIND_DREAM_SERVICE': 1,
 'BIND_GET_INSTALL_REFERRER_SERVICE': 1,
 'BIND_INPUT_METHOD': 1,
 'BIND_NFC_SERVICE': 1,
 'BIND_0TIFICATION_LISTENER_SERVICE': 1,
 'B

В каждом столбце есть по одному пропуску.<br>Посмотрим на строку, содержащую пропуск по первой фиче.

In [5]:
data[data['ACCESS_ALL_DOWNLOADS'].isna()]

,ACCESS_ALL_DOWNLOADS,ACCESS_CACHE_FILESYSTEM,ACCESS_CHECKIN_PROPERTIES,ACCESS_COARSE_LOCATION,ACCESS_COARSE_UPDATES,ACCESS_FINE_LOCATION,ACCESS_LOCATION_EXTRA_COMMANDS,ACCESS_MOCK_LOCATION,ACCESS_MTK_MMHW,ACCESS_NETWORK_STATE,...,Landroid/telephony/TelephonyManager;->getLine1Number,Landroid/telephony/TelephonyManager;->getNetworkOperator,Landroid/telephony/TelephonyManager;->getNetworkOperatorName,Landroid/telephony/TelephonyManager;->getNetworkCountryIso,Landroid/telephony/TelephonyManager;->getSimOperator,Landroid/telephony/TelephonyManager;->getSimOperatorName,Landroid/telephony/TelephonyManager;->getSimCountryIso,Landroid/telephony/TelephonyManager;->getSimSerialNumber,Lorg/apache/http/impl/client/DefaultHttpClient;->execute,Label
2533,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Нам повезло, в этой строке и находятся все пропуски, так что удалим ее.

In [6]:
data = data.drop(index=data[data['ACCESS_ALL_DOWNLOADS'].isna()].index)

Так как данные у нас - флаги, принимающие значения 0 или 1, есть смысл привести все к типу bool и удалить все константные флаги.

Небольшой спойлер: если у вас появляется вопрос, почему сначала была удалена строка с пропусками, ответ очень прост - иначе при приведении к типу бул она превратится в строку, полностью состояющую из True значений, баг это или фича - риторический вопрос.

In [7]:
data = data.astype(bool)
data = data.drop(columns = [i for i in data.columns if data[i].max()==data[i].min()])
data.shape

(4464, 200)

Таким образом, после удаления фичей-констант, из 241 исходной фичи остались 200.

Что, в общем-то, все еще много и стоит подумать об отборе наиболее важных из них.

Еще один вариант - перейти к "новым фичам", применив Метод Главных Компонент (PCA), но так как все фичи бинарные, это, честно говоря, сомнительная затея, так как компоненты в МГК являются линейными комбинациями исходных фич.

Поэтому рабочий вариант - отобрать наиболее важные (то есть коррелирующие с таргетом).

Посчитаем корреляцию фичей с таргетом, выведем топ-25.

In [8]:
top_25_features = data.corr().abs()['Label'].sort_values(ascending=False).index[1:].tolist()[:25]
top_25_features

['RECEIVE_BOOT_COMPLETED',
 'Ljava/net/URL;->openConnection',
 'Landroid/location/LocationManager;->getLastK0wnLocation',
 'GET_TASKS',
 'Ljava/lang/System;->load',
 'WAKE_LOCK',
 'Ljava/lang/System;->loadLibrary',
 'Ldalvik/system/DexClassLoader;->loadClass',
 'KILL_BACKGROUND_PROCESSES',
 'Landroid/telephony/TelephonyManager;->getSimOperatorName',
 'Landroid/telephony/TelephonyManager;->getSimOperator',
 'Landroid/telephony/TelephonyManager;->getNetworkCountryIso',
 'Landroid/telephony/TelephonyManager;->getNetworkOperator',
 'Landroid/content/pm/PackageManager;->getInstalledPackages',
 'SYSTEM_ALERT_WINDOW',
 'ACCESS_FINE_LOCATION',
 'VIBRATE',
 'ACCESS_COARSE_LOCATION',
 'DISABLE_KEYGUARD',
 'Landroid/telephony/TelephonyManager;->getCellLocation',
 'Landroid/telephony/TelephonyManager;->getNetworkOperatorName',
 'Landroid/telephony/TelephonyManager;->getSimCountryIso',
 'WRITE_EXTERNAL_STORAGE',
 'CHANGE_NETWORK_STATE',
 'Landroid/media/AudioRecord;->startRecording']

С первичным анализом данных можно закончить, пора и модельки зафиттить))

Импортируем необходимые для обучения и скоринга моделей классы.<br>
Сравнивать будем модели на основе ЛогРегрессии, Дерева решений, Случайного леса, Дерева с бустингом.<br>
Для более объективного сравнения будем использовать кросс-валидацию на 5 фолдов, так как при обычном сплите на трейн-тест можно получить очень удачное или очень неудачное разбиение.<br>
Также проверим, можно ли использовать топ-25 наиболее коррелирующих с таргетом фичей.

In [9]:
from sklearn.model_selection import KFold
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier, XGBRFClassifier
from lightgbm import LGBMClassifier

import joblib

Как и упомяналось ранее, классы не сбалансированы, поэтому использовать accuracy в качестве метрики для выбора модели не стоит, используем f1-score.
<br>Для оптимизации гиперпараметров используем HalvingSearch по сетке с кросс-валидацией на 5 фолдов.

Сначала определим модели и сетки параметров для подбора.

In [10]:
models = {'GradientBoostingClassifier': GradientBoostingClassifier(),
          'RandomForestClassifier':     RandomForestClassifier(),
          'LogisticRegression':         LogisticRegression(),
          'SVC':                        SVC(),
          'DecisionTreeClassifier':     DecisionTreeClassifier(),
          'XGBClassifier':              XGBClassifier(),
          'XGBRFClassifier':            XGBRFClassifier(),
          'LGBMClassifier':             LGBMClassifier(objective='binary')
}

parameters_grids = {
    'GradientBoostingClassifier': {
        'learning_rate':     [0.1, 0.01, 0.001],
        'n_estimators':      [100, 200, 500],
        'max_depth':         [3, 5, 10],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf':  [1, 2, 3],
        'max_features':      ['sqrt', 'log2']
    },
    'RandomForestClassifier': {
        'n_estimators':      [100, 200, 500],
        'max_depth':         [None, 5, 10],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf':  [1, 2, 3],
        'max_features':      ['sqrt', 'log2'],
        'bootstrap':         [True, False],
        'criterion':         ['gini', 'entropy']
    },
    'LogisticRegression': {
        'penalty': ['l1', 'l2'],
        'C':       [0.1, 1, 10],
        'solver':  ['liblinear', 'saga']
    },
    'SVC': {
        'C':      [0.1, 1, 10],
        'kernel': ['linear', 'rbf', 'sigmoid'],
        'gamma':  ['scale', 'auto']
    },
    'DecisionTreeClassifier': {
        'max_depth':         [None, 5, 10],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf':  [1, 2, 3],
        'max_features':      ['sqrt', 'log2'],
        'criterion':         ['gini', 'entropy']
    },
    'XGBClassifier': {
        'learning_rate':    [0.1, 0.01, 0.001],
        'n_estimators':     [100, 200, 500],
        'max_depth':        [3, 5, 10],
        'subsample':        [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    },
    'XGBRFClassifier': {
        'learning_rate':    [0.1, 0.01, 0.001],
        'n_estimators':     [100, 200, 500],
        'max_depth':        [3, 5, 10],
        'subsample':        [0.8, 1.0],
        'colsample_bynode': [0.8, 1.0]
    },
    'LGBMClassifier': {
        'learning_rate':    [0.1, 0.01, 0.001],
        'n_estimators':     [100, 200, 500],
        'max_depth':        [3, 5, 10],
        'subsample':        [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0],
        'reg_alpha':        [0.0, 0.1, 0.5],
        'reg_lambda':       [0.0, 0.1, 0.5]
    }
}

In [ ]:
features, target = data.drop(columns='Label'), data['Label']

scores = []

for model_name in models:
    searcher = HalvingGridSearchCV(estimator = models[model_name].set_params(random_state = 42), 
                                   param_grid = parameters_grids[model_name], 
                                   factor = 3, 
                                   scoring = 'f1',
                                   min_resources = 'smallest',
                                   cv = KFold(n_splits=5, shuffle=True, random_state=42), 
                                   n_jobs = -1).fit(features,target)

    scores += [{'model': model_name, 
                'f1': searcher.best_score_, 
                'params': searcher.best_params_}]

Просмотрим результаты: модели, f1-меру и наборы гиперпараметров.

In [12]:
scores_df = pd.DataFrame(scores).sort_values('f1', ascending=False)
scores_df

,model,f1,params
1,RandomForestClassifier,0.995264,"{'bootstrap': False, 'criterion': 'entropy', '..."
0,GradientBoostingClassifier,0.994254,"{'learning_rate': 0.1, 'max_depth': 10, 'max_f..."
5,XGBClassifier,0.992387,"{'colsample_bytree': 0.8, 'learning_rate': 0.1..."
7,LGBMClassifier,0.990154,"{'colsample_bytree': 0.8, 'learning_rate': 0.1..."
3,SVC,0.989348,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}"
2,LogisticRegression,0.971932,"{'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}"
4,DecisionTreeClassifier,0.967874,"{'criterion': 'gini', 'max_depth': 5, 'max_fea..."
6,XGBRFClassifier,0.884465,"{'colsample_bynode': 0.8, 'learning_rate': 0.0..."


Повторим обучение и скоринг, используя топ-25 фич.

In [ ]:
features, target = data[top_25_features], data['Label']

scores = []

for model_name in models:
    searcher = HalvingGridSearchCV(estimator = models[model_name].set_params(random_state = 42), 
                                   param_grid = parameters_grids[model_name], 
                                   factor = 3, 
                                   scoring = 'f1',
                                   min_resources = 'smallest',
                                   cv = KFold(n_splits=5, shuffle=True, random_state=42), 
                                   n_jobs = -1).fit(features,target)

    scores += [{'model': model_name, 
                'f1': searcher.best_score_, 
                'params': searcher.best_params_}]

Также просмотрим результаты: модели, f1-меру и наборы гиперпараметров.

In [12]:
scores_df_25 = pd.DataFrame(scores).sort_values('f1', ascending=False)
scores_df_25

,model,f1,params
0,GradientBoostingClassifier,0.994221,"{'learning_rate': 0.01, 'max_depth': 10, 'max_..."
1,RandomForestClassifier,0.987136,"{'bootstrap': True, 'criterion': 'gini', 'max_..."
5,XGBClassifier,0.985879,"{'colsample_bytree': 0.8, 'learning_rate': 0.1..."
7,LGBMClassifier,0.984628,"{'colsample_bytree': 0.8, 'learning_rate': 0.0..."
3,SVC,0.982292,"{'C': 0.1, 'gamma': 'auto', 'kernel': 'linear'}"
4,DecisionTreeClassifier,0.971740,"{'criterion': 'entropy', 'max_depth': 5, 'max_..."
2,LogisticRegression,0.967333,"{'C': 10, 'penalty': 'l1', 'solver': 'saga'}"
6,XGBRFClassifier,0.891114,"{'colsample_bynode': 1.0, 'learning_rate': 0.0..."


В целом все модели успешно обучились и показали хорошие результаты.

При этом удалось получить практически идентичные результаты, используя как 25 фич, так и все.

Сохраним топ-3 модели, обучив на всем датасете, используя 25 фич с наибольшим коэффициентом корреляции с таргетом.

In [13]:
for score in scores_df_25.values[:3]:
    model = models[score[0]].set_params(random_state = 42, **score[2])
    model.fit(features, target)
    joblib.dump(model, f'./models/{score[0]}.joblib')